<h1>Proyecto ETL</h1>

<h2>Presentación</h2>

<p>La idea de este proyecto es realizar un proceso basico de extraer, transformar y cargar para practicar mi forma de documentar y usar github así como demostrar conocimiento basico de request, pandas y SQL. </p>

In [1]:
import requests
import random
import pandas as pd
from sqlalchemy import create_engine

<p>Encontre esta api que me parecio interesante y facil de usar asi que voy a investigarla usando postman y la documentacion</p>

http://universities.hipolabs.com/
https://github.com/Hipo/university-domains-list-api

<p>Me aseguro de que funcione en jupyter</p>

In [2]:
url_api = "http://universities.hipolabs.com/search"


respuesta = requests.get(url_api)

#compruebo de que funcione, caso contrario que imprima el codigo de error.
if respuesta.status_code == 200:
    universidades = respuesta.json()
    #A modo de ejemplo elijo una universidad al azar y la imprimo para asegurarme de que funcione
    universidadejemplo = random.choice(universidades)
    print(universidadejemplo)
else:
    print('La solicitud no fue exitosa. Código de estado:', respuesta.status_code)

{'name': 'Ecole Française de Papeterie et des Industries Graphiques', 'country': 'France', 'state-province': None, 'domains': ['efpg.inpg.fr'], 'alpha_two_code': 'FR', 'web_pages': ['http://www.efpg.inpg.fr/']}


<p>La api devuelve una lista de universidades en .json, para reducir la lista voy pegarle al endpoint que devuelve solo las de Argentina</p>

http://universities.hipolabs.com/search?country=Argentina

<h2>Extraer</h2>

<p>Voy a definir una funcion extrer que va a guardar los datos en formato .json</p>

In [3]:
def extraer():
    """ Esta api extrae datos desde
    http://universities.hipolabs.com/
    """
    url = 'http://universities.hipolabs.com/search?country=Argentina'
    datos = requests.get(url).json()
    return datos

<h2>Transformar</h2>

<p>Voy a realizar una transformacion antes de guardar los datos, usando pandas</p>

In [4]:
df = pd.DataFrame(extraer())
df

,name,country,state-province,domains,alpha_two_code,web_pages
0,Universidad Atlantida Argentina,Argentina,Buenos Aires,[atlantida.edu.ar],AR,[http://www.atlantida.edu.ar/]
1,Universidad Austral Buenos Aires,Argentina,Buenos Aires,[austral.edu.ar],AR,[http://www.austral.edu.ar/]
2,"Universidad CAECE, Buenos Aires",Argentina,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],AR,[http://www.caece.edu.ar/]
3,Instituto Universitario CEMA,Argentina,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],AR,[http://www.cema.edu.ar/]
4,Instituto de Enseñanza Superior del Ejército,Argentina,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],AR,[http://www.iese.edu.ar/]
...,...,...,...,...,...,...
81,Universidad Nacional de Tres de Febrero,Argentina,Buenos Aires,[untref.edu.ar],AR,[http://www.untref.edu.ar/]
82,Universidad Torcuato di Tella,Argentina,Ciudad Autónoma de Buenos Aires,[utdt.edu],AR,[http://www.utdt.edu/]
83,Universidad Tecnológica Nacional,Argentina,Ciudad Autónoma de Buenos Aires,[utn.edu.ar],AR,[http://www.utn.edu.ar/]
84,Universidad Abierta Interamericana,Argentina,Ciudad Autónoma de Buenos Aires,[vaneduc.edu.ar],AR,[http://www.vaneduc.edu.ar/uai/]


In [5]:
df.describe()

,name,country,state-province,domains,alpha_two_code,web_pages
count,86,86,65,86,86,86
unique,86,1,17,86,1,86
top,Universidad Atlantida Argentina,Argentina,Buenos Aires,[atlantida.edu.ar],AR,[http://www.atlantida.edu.ar/]
freq,1,86,20,1,86,1


<p>Verifico parte de la estructura del dataframe, no tiene inconvenientes respecto a la "limpieza" de los datos, solamente queda filtrarlos, tener en cuenta que ciudad y provincia de buenos aires aparecen separados</p>

In [6]:
df = df.drop(columns=['alpha_two_code','country'])

<p>Ya que por medio de la api filtre a las universidades argentinas, resultan redundantes ambas columnas</p>

In [7]:
df = df.loc[(df["state-province"] == "Buenos Aires")| (df["state-province"] == "Ciudad Autónoma de Buenos Aires")]
df.head()

,name,state-province,domains,web_pages
0,Universidad Atlantida Argentina,Buenos Aires,[atlantida.edu.ar],[http://www.atlantida.edu.ar/]
1,Universidad Austral Buenos Aires,Buenos Aires,[austral.edu.ar],[http://www.austral.edu.ar/]
2,"Universidad CAECE, Buenos Aires",Ciudad Autónoma de Buenos Aires,[caece.edu.ar],[http://www.caece.edu.ar/]
3,Instituto Universitario CEMA,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],[http://www.cema.edu.ar/]
4,Instituto de Enseñanza Superior del Ejército,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],[http://www.iese.edu.ar/]


<p>Ahora quedaria reorganizar las columnas, convertir listas en cadenas y restablecer el indice<p/>

In [9]:
df = df[['name', 'state-province', 'domains', 'web_pages']]
df['domains'] = df['domains'].apply(lambda x: ', '.join(x))
df['web_pages'] = df['web_pages'].apply(lambda x: ', '.join(x))
df = df.reset_index(drop=True)

<p>Ahora lo voy a cargar a una base de datos relacional de forma local con SqlAlchemy</p>

In [10]:
def cargar(df:pd.DataFrame):
    """ carga los datos en una base de datos sqllite """
    disk_engine = create_engine('sqlite:///bdatos.db')
    df.to_sql('cal_uni', disk_engine, if_exists='replace')

In [11]:
cargar(df)

<p>L<p/>